# Importar parametros

In [ ]:
import sys

archivojson=sys.argv[1]

In [ ]:
import json

with open(archivojson, 'r') as file:
    json_arch =json.load(file)

# Iniciar Spark

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat

In [5]:
spark = SparkSession.builder \
    .appName("Conversion 1") \
    .config("spark.scheduler.mode", "FAIR") \
    .getOrCreate()


In [145]:
# Enable Adaptive Query Execution
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.adaptive.optimizerEnabled", "true")

In [167]:
spark.conf.set("spark.sql.shuffle.partitions", "200")

In [168]:
print('spark.sql.shuffle.partitions')

spark.sql.shuffle.partitions


In [ ]:
#Ejemplo
executor_cores = spark.conf.get("spark.executor.cores")
print(f"Número de núcleos del ejecutor: {executor_cores}")

#Ejemplo
driver_cores = spark.conf.get("spark.driver.cores")
print(f"Número de núcleos del driver: {driver_cores}")

In [ ]:
#EJEMPLO
spark.conf.set("spark.executor.cores", "8")
spark.conf.set("spark.executor.memory", "32g")

In [ ]:
#EJEMPLO
spark.conf.set("spark.driver.cores", "16")
spark.conf.set("spark.driver.memory", "128g")

In [ ]:
# Ejemplo de lectura de archivo de parametros
googlep=spark.read.csv(json_arch["Archivo2"])

In [8]:
# Lectura de Archivos de manera directa por csv
googlep = spark.read.csv("googleplaystore.csv", header=True, inferSchema=True)

# Uso de Spark Dataframe

In [9]:
googlep.show()

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|      Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design|   January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|  January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|    August 1, 2018|             1.2.4|4.0.3 

In [10]:
googlep.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [20]:
googlecast=googlep.select('App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver', col("Rating").cast("int"))
googlecast.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)
 |-- Rating: integer (nullable = true)



In [31]:
genresgoog=googlecast.groupBy('Genres').count()
genresgoog.collect()

[Row(Genres='Video Players & Editors;Creativity', count=2),
 Row(Genres='Adventure;Action & Adventure', count=13),
 Row(Genres='Education', count=549),
 Row(Genres='Trivia', count=38),
 Row(Genres='Art & Design;Action & Adventure', count=2),
 Row(Genres='Auto & Vehicles', count=85),
 Row(Genres='Travel & Local;Action & Adventure', count=1),
 Row(Genres='Simulation;Action & Adventure', count=11),
 Row(Genres='Education;Pretend Play', count=23),
 Row(Genres='Tools;Education', count=1),
 Row(Genres='Entertainment', count=623),
 Row(Genres='Education;Education', count=50),
 Row(Genres='Entertainment;Music & Video', count=27),
 Row(Genres='Parenting;Brain Games', count=1),
 Row(Genres='Simulation;Pretend Play', count=4),
 Row(Genres='Arcade;Action & Adventure', count=16),
 Row(Genres='Educational;Creativity', count=5),
 Row(Genres='Arcade;Pretend Play', count=1),
 Row(Genres='Casual;Music & Video', count=2),
 Row(Genres='Adventure', count=75),
 Row(Genres='Arcade', count=220),
 Row(Genres='

In [37]:
googleconc=googlecast.select(concat(col('App'), col('Genres')).alias('App_Genres'))
googleconc.show()

+--------------------+
|          App_Genres|
+--------------------+
|Photo Editor & Ca...|
|Coloring book moa...|
|U Launcher Lite –...|
|Sketch - Draw & P...|
|Pixel Draw - Numb...|
|Paper flowers ins...|
|Smoke Effect Phot...|
|Infinite PainterA...|
|Garden Coloring B...|
|Kids Paint Free -...|
|Text on Photo - F...|
|Name Art Photo Ed...|
|Tattoo Name On My...|
|Mandala Coloring ...|
|3D Color Pixel by...|
|Learn To Draw Kaw...|
|Photo Designer - ...|
|350 Diy Room Deco...|
|FlipaClip - Carto...|
|ibis Paint XArt &...|
+--------------------+
only showing top 20 rows



In [82]:
goog=googlep.drop( 'Reviews', 'Size', 'Installs', 'Type', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver')

In [83]:
goog.show()

+--------------------+--------------+------+-----+
|                 App|      Category|Rating|Price|
+--------------------+--------------+------+-----+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    0|
| Coloring book moana|ART_AND_DESIGN|   3.9|    0|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|    0|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5|    0|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    0|
|Paper flowers ins...|ART_AND_DESIGN|   4.4|    0|
|Smoke Effect Phot...|ART_AND_DESIGN|   3.8|    0|
|    Infinite Painter|ART_AND_DESIGN|   4.1|    0|
|Garden Coloring Book|ART_AND_DESIGN|   4.4|    0|
|Kids Paint Free -...|ART_AND_DESIGN|   4.7|    0|
|Text on Photo - F...|ART_AND_DESIGN|   4.4|    0|
|Name Art Photo Ed...|ART_AND_DESIGN|   4.4|    0|
|Tattoo Name On My...|ART_AND_DESIGN|   4.2|    0|
|Mandala Coloring ...|ART_AND_DESIGN|   4.6|    0|
|3D Color Pixel by...|ART_AND_DESIGN|   4.4|    0|
|Learn To Draw Kaw...|ART_AND_DESIGN|   3.2|    0|
|Photo Designer - ...|ART_AND_D

In [123]:
google = googlecast.join(goog, googlecast.App == goog.App, 'left')
                                                                    
google.show()

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+------+--------------------+--------------+------+-----+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|      Last Updated|       Current Ver| Android Ver|Rating|                 App|      Category|Rating|Price|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+------+--------------------+--------------+------+-----+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design|   January 7, 2018|             1.0.0|4.0.3 and up|     4|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    0|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Des

# Creacion y uso de Vistas temporales

In [124]:
google.createOrReplaceTempView('temp_google')

In [128]:
spark.sql('SELECT * FROM temp_google LIMIT 5 ').show()

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+------+--------------------+--------------+------+-----+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|    Last Updated|       Current Ver| Android Ver|Rating|                 App|      Category|Rating|Price|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+------+--------------------+--------------+------+-----+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design| January 7, 2018|             1.0.0|4.0.3 and up|     4|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    0|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret

In [132]:
googlep.createOrReplaceTempView('df_original')

In [137]:
spark.sql('SELECT App, Price AS Precio, Rating AS Puntaje FROM df_original LIMIT 11').show()

+--------------------+------+-------+
|                 App|Precio|Puntaje|
+--------------------+------+-------+
|Photo Editor & Ca...|     0|    4.1|
| Coloring book moana|     0|    3.9|
|U Launcher Lite –...|     0|    4.7|
|Sketch - Draw & P...|     0|    4.5|
|Pixel Draw - Numb...|     0|    4.3|
|Paper flowers ins...|     0|    4.4|
|Smoke Effect Phot...|     0|    3.8|
|    Infinite Painter|     0|    4.1|
|Garden Coloring Book|     0|    4.4|
|Kids Paint Free -...|     0|    4.7|
|Text on Photo - F...|     0|    4.4|
+--------------------+------+-------+



In [149]:
sqlunion=spark.sql('SELECT Category, SUM(Price) AS Precio FROM df_original  GROUP BY Category LIMIT 11').show()

+------------------+------+
|          Category|Precio|
+------------------+------+
|            EVENTS|   0.0|
|            COMICS|   0.0|
|            SPORTS|   0.0|
|           WEATHER|   0.0|
|     VIDEO_PLAYERS|   0.0|
| AUTO_AND_VEHICLES|   0.0|
|         PARENTING|   0.0|
|     ENTERTAINMENT|   0.0|
|   PERSONALIZATION|   0.0|
|HEALTH_AND_FITNESS|   0.0|
|  TRAVEL_AND_LOCAL|   0.0|
+------------------+------+



# JOIN ENTRE DATAFRAME Y VISTA TEMPORAL

In [165]:
temp_df=googlep.join(spark.table("df_original"), googlep.App == spark.table("df_original").App, "inner").show()

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|      Last Updated|       Current Ver| Android Ver|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|      Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------

In [ ]:
temp_df.write.mode('Append').saveAsTable('database.table')
temp_df.write.mode('overwrite').insertInto('database.table2')